In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
# from keras.applications import ResNet18
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam

2024-02-25 18:45:26.516553: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-25 18:45:26.542613: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-25 18:45:26.893304: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from keras.applications import ResNet50

In [4]:
dataset_dir = "../dataset_output/google_asl/imgs/"

In [5]:
img_size = (224, 224)
batch_size = 64

In [6]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3,  # Splitting data into train and validation
)

In [7]:
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Training data subset
)

Found 18511 images belonging to 70 classes.


In [8]:
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Validation data subset
)

Found 7892 images belonging to 70 classes.


In [9]:
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

2024-02-25 18:45:34.176571: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 18:45:34.194017: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 18:45:34.194180: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 18:45:34.196244: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 18:45:34.196378: I tensorflow/compile

In [10]:
# Freeze the convolutional base
for layer in resnet.layers:
    layer.trainable = False

In [11]:
model = Sequential([
    resnet,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(70, activation='softmax')  # 70 output classes
])

In [12]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/100


2024-02-25 18:45:41.332266: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-02-25 18:45:41.441266: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied
2024-02-25 18:45:41.807729: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-02-25 18:45:42.489624: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x88f30d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-25 18:45:42.489654: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2024-02-25 18:45:42.493055: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-25 18:45:42.569133: I tensorfl

289/289 [==============================] - 75s 250ms/step - loss: 4.3862 - accuracy: 0.0141 - val_loss: 4.2479 - val_accuracy: 0.0158
Epoch 2/100
289/289 [==============================] - 72s 249ms/step - loss: 4.2500 - accuracy: 0.0145 - val_loss: 4.2475 - val_accuracy: 0.0158
Epoch 3/100
289/289 [==============================] - 70s 243ms/step - loss: 4.2478 - accuracy: 0.0152 - val_loss: 4.2472 - val_accuracy: 0.0158
Epoch 4/100
289/289 [==============================] - 71s 245ms/step - loss: 4.2476 - accuracy: 0.0154 - val_loss: 4.2471 - val_accuracy: 0.0158
Epoch 5/100
289/289 [==============================] - 76s 265ms/step - loss: 4.2476 - accuracy: 0.0142 - val_loss: 4.2471 - val_accuracy: 0.0155
Epoch 6/100
289/289 [==============================] - 74s 256ms/step - loss: 4.2475 - accuracy: 0.0157 - val_loss: 4.2470 - val_accuracy: 0.0158
Epoch 7/100
289/289 [==============================] - 70s 243ms/step - loss: 4.2474 - accuracy: 0.0150 - val_loss: 4.2470 - val_accurac

Epoch 57/100
289/289 [==============================] - 72s 250ms/step - loss: 4.2474 - accuracy: 0.0157 - val_loss: 4.2469 - val_accuracy: 0.0158
Epoch 58/100
289/289 [==============================] - 72s 250ms/step - loss: 4.2474 - accuracy: 0.0152 - val_loss: 4.2470 - val_accuracy: 0.0158
Epoch 59/100
289/289 [==============================] - 69s 240ms/step - loss: 4.2474 - accuracy: 0.0153 - val_loss: 4.2469 - val_accuracy: 0.0158
Epoch 60/100
289/289 [==============================] - 71s 245ms/step - loss: 4.2474 - accuracy: 0.0149 - val_loss: 4.2470 - val_accuracy: 0.0156
Epoch 61/100
289/289 [==============================] - 71s 245ms/step - loss: 4.2474 - accuracy: 0.0153 - val_loss: 4.2470 - val_accuracy: 0.0158
Epoch 62/100
289/289 [==============================] - 76s 262ms/step - loss: 4.2474 - accuracy: 0.0157 - val_loss: 4.2470 - val_accuracy: 0.0158
Epoch 63/100
289/289 [==============================] - 72s 248ms/step - loss: 4.2474 - accuracy: 0.0154 - val_loss: 4

In [1]:
0

0